### Import required modules and defining any utility functions if we need them

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import r2_score as r2, mean_squared_error as mse
import json

__Set float_format preference option for display to show max 3 values after decimal__

__Improves Readability__

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

__Utility functions for creating a backup of dataframe and load from backup if required. In case something fails or doesn't go as expected, I can load my DFs from this backup and won't have to re-run the whole Notebook everytime.__

In [22]:
import os
from datetime import datetime as dt
backup_file = "../dataset/backup.feather"
train_file = "../dataset/train_df.feather"

def write_df_to_feather(df, backup_df, overwrite=True):
    backup_files = [train_file, backup_file]
    if overwrite:
        for f in [train_file, backup_file]:
            if os.path.exists(f):
                os.remove(f)
    else:
        for f in [train_file, backup_file]:
            if os.path.exists(f):
                if not new_f:
                    new_f = str(dt.now()).join(os.path.splitext(f))
                os.rename(f, new_f)
                backup_files.append(new_f)
    df.to_feather(train_file)
    backup_df.to_feather(backup_file)
    return backup_files[-2:]       #


### Load CSVs

In [3]:
train_df = pd.read_csv('../dataset/train.csv', dtype={
                                                    'fullVisitorId': str, 
                                                    'visitId': str})
test_df = pd.read_csv('../dataset/test.csv', dtype={
                                                    'fullVisitorId': str, 
                                                    'visitId': str})

__JSON values will have to be expanded into columns for further processing__

In [4]:
def expand_json_to_columns(df, column):
    df[column] = df[column].apply(json.loads)
    json_df = json_normalize(df[column])
    json_df.columns = ["{}-{}".format(column, subcolumn) for subcolumn in json_df.columns]
    df.drop(column, axis=1, inplace=True)
    df = df.merge(json_df, right_index=True, left_index=True)
    
    print("New Shape of df: {}".format(str(df.shape)))
    return df

In [5]:
def convert_date_and_visitStartTime(df):
    df['date'] = pd.to_datetime(df.date, format="%Y%m%d")
    df['visitStartTime'] = pd.to_datetime(df.visitStartTime, unit='s')
    return df

In [6]:
train_df.columns

Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'socialEngagementType', 'totals', 'trafficSource',
       'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')

In [8]:
columns = ['device', 'geoNetwork', 'totals', 'trafficSource']

for column in columns: 
    train_df = expand_json_to_columns(train_df, column)
    test_df = expand_json_to_columns(test_df, column)

New Shape of df: (903653, 27)
New Shape of df: (804684, 27)
New Shape of df: (903653, 37)
New Shape of df: (804684, 37)
New Shape of df: (903653, 42)
New Shape of df: (804684, 41)
New Shape of df: (903653, 55)
New Shape of df: (804684, 53)


In [9]:
train_df = convert_date_and_visitStartTime(train_df)
test_df = convert_date_and_visitStartTime(test_df)

In [10]:
columns = train_df.columns
display(columns); display(train_df.head()); display(train_df.describe(include='all'))

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device-browser', 'device-browserSize', 'device-browserVersion',
       'device-deviceCategory', 'device-flashVersion', 'device-isMobile',
       'device-language', 'device-mobileDeviceBranding',
       'device-mobileDeviceInfo', 'device-mobileDeviceMarketingName',
       'device-mobileDeviceModel', 'device-mobileInputSelector',
       'device-operatingSystem', 'device-operatingSystemVersion',
       'device-screenColors', 'device-screenResolution', 'geoNetwork-city',
       'geoNetwork-cityId', 'geoNetwork-continent', 'geoNetwork-country',
       'geoNetwork-latitude', 'geoNetwork-longitude', 'geoNetwork-metro',
       'geoNetwork-networkDomain', 'geoNetwork-networkLocation',
       'geoNetwork-region', 'geoNetwork-subContinent', 'totals-bounces',
       'totals-hits', 'totals-newVisits', 'totals-pageviews',
       'totals-transactionRevenue

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device-browser,device-browserSize,...,trafficSource-adwordsClickInfo.isVideoAd,trafficSource-adwordsClickInfo.page,trafficSource-adwordsClickInfo.slot,trafficSource-campaign,trafficSource-campaignCode,trafficSource-isTrueDirect,trafficSource-keyword,trafficSource-medium,trafficSource-referralPath,trafficSource-source
0,Organic Search,2016-09-02,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,2016-09-02 15:33:05,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,2016-09-02,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,2016-09-03 05:22:27,Firefox,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,2016-09-02,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,2016-09-03 01:16:26,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,2016-09-02,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,2016-09-03 05:40:13,UC Browser,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,2016-09-02,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,2016-09-02 13:23:20,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device-browser,device-browserSize,...,trafficSource-adwordsClickInfo.isVideoAd,trafficSource-adwordsClickInfo.page,trafficSource-adwordsClickInfo.slot,trafficSource-campaign,trafficSource-campaignCode,trafficSource-isTrueDirect,trafficSource-keyword,trafficSource-medium,trafficSource-referralPath,trafficSource-source
count,903653,903653,903653,903653,903653,903653,903653.000,903653,903653,903653,...,21460,21460,21460,903653,1,274005,400724,903653,330941,903653
unique,8,366,714167,902755,1,886303,nan,887159,54,1,...,1,8,2,10,1,1,3659,7,1475,380
top,Organic Search,2016-11-28 00:00:00,1957458976293878100,1087285358537852421_1491116335,Not Socially Engaged,1493146175,nan,2017-04-25 18:49:35,Chrome,not available in demo dataset,...,False,1,Top,(not set),11251kjhkvahf,True,(not provided),organic,/,google
freq,381561,4807,278,2,903653,8,nan,8,620364,903653,...,21460,21362,20956,865347,1,274005,366363,381561,75523,400788
first,NaN,2016-08-01 00:00:00,NaN,NaN,NaN,NaN,nan,2016-08-01 07:00:12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2017-08-01 00:00:00,NaN,NaN,NaN,NaN,nan,2017-08-02 06:59:53,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2.265,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,9.284,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Let the cleaning begin

__Extract columns with only one unique value in them. We'll explore these further and drop them if nothing can be done since non-changing values won't contribute to the prediction.__

In [16]:
unique_value_columns = [col for col in train_df.columns if train_df[col].nunique()==1]
display(train_df[unique_value_columns].describe(include='all'))

,socialEngagementType,device-browserSize,device-browserVersion,device-flashVersion,device-language,device-mobileDeviceBranding,device-mobileDeviceInfo,device-mobileDeviceMarketingName,device-mobileDeviceModel,device-mobileInputSelector,...,geoNetwork-latitude,geoNetwork-longitude,geoNetwork-networkLocation,totals-bounces,totals-newVisits,totals-visits,trafficSource-adwordsClickInfo.criteriaParameters,trafficSource-adwordsClickInfo.isVideoAd,trafficSource-campaignCode,trafficSource-isTrueDirect
count,903653,903653,903653,903653,903653,903653,903653,903653,903653,903653,...,903653,903653,903653,450630,703060,903653,903653,21460,1,274005
unique,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
top,Not Socially Engaged,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,not available in demo dataset,False,11251kjhkvahf,True
freq,903653,903653,903653,903653,903653,903653,903653,903653,903653,903653,...,903653,903653,903653,450630,703060,903653,903653,21460,1,274005


In [20]:
# Values that look of no use at first glance. Let's remove these and we'll explore the remaining futher. 
# Criteria for removal: unique value in the column i.e nunique == 1, count == 903653 i.e. no null values
drop_columns = [col for col in train_df.columns if (train_df[col].nunique()==1) and (train_df[col].count() == 903653)]
r_unique_value_columns = list(set(unique_value_columns).difference(set(drop_columns)))
print(f"Number of columns dropped: {len(drop_columns)}"); print(f"Number of columns kept: {len(r_unique_value_columns)}")

Number of columns dropped: 19
Number of columns kept: 5


In [23]:
backup = train_df[drop_columns]
train_df.drop(drop_columns, axis=1, inplace=True)
backup_files = write_df_to_feather(train_df, backup)

In [24]:
train_df.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device-browser,device-deviceCategory,device-isMobile,...,trafficSource-adwordsClickInfo.isVideoAd,trafficSource-adwordsClickInfo.page,trafficSource-adwordsClickInfo.slot,trafficSource-campaign,trafficSource-campaignCode,trafficSource-isTrueDirect,trafficSource-keyword,trafficSource-medium,trafficSource-referralPath,trafficSource-source
0,Organic Search,2016-09-02,1131660440785968503,1131660440785968503_1472830385,1472830385,1,2016-09-02 15:33:05,Chrome,desktop,False,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,2016-09-02,377306020877927890,377306020877927890_1472880147,1472880147,1,2016-09-03 05:22:27,Firefox,desktop,False,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,2016-09-02,3895546263509774583,3895546263509774583_1472865386,1472865386,1,2016-09-03 01:16:26,Chrome,desktop,False,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,2016-09-02,4763447161404445595,4763447161404445595_1472881213,1472881213,1,2016-09-03 05:40:13,UC Browser,desktop,False,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,2016-09-02,27294437909732085,27294437909732085_1472822600,1472822600,2,2016-09-02 13:23:20,Chrome,mobile,True,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


__Taking a closer look at the columns we decided to keep__

In [21]:
train_df[r_unique_value_columns].describe(include='all')

,trafficSource-campaignCode,totals-newVisits,trafficSource-isTrueDirect,totals-bounces,trafficSource-adwordsClickInfo.isVideoAd
count,1,703060,274005,450630,21460
unique,1,1,1,1,1
top,11251kjhkvahf,1,True,1,False
freq,1,703060,274005,450630,21460
